In [1]:
%matplotlib inline
from IPython.display import display
import ast

import pandas as pd
pd.set_option('max_colwidth',100)
import numpy as np
import statsmodels.api as sm
from scipy import stats

In [2]:
whichyear='both' 
suff = 'V2_60pred_julaug'
letters=['a','b','c','d','e','f','g','h','i','j','k','l','m']
modelnames=['log chla','log cybv','log mic','log TN','log TP']

df_chla = pd.DataFrame.from_csv('df_chla_{}_{}.csv'.format(whichyear,suff))
df_cybv = pd.DataFrame.from_csv('df_cybv_{}_{}.csv'.format(whichyear,suff))
df_mic = pd.DataFrame.from_csv('df_mic_{}_{}.csv'.format(whichyear,suff))
df_clean = pd.DataFrame.from_csv('df_clean_{}_{}.csv'.format(whichyear,suff))
for df in [df_clean,df_mic,df_chla,df_cybv]:
    df.reset_index(inplace=True)
    df.set_index(['SITE_ID','VISIT_NO'],inplace=True)
    
dict_meta = {
    'chla' :[df_chla,'chla_log',4],
    'cybv' :[df_cybv,'cybv_log',4],
    'mic'  :[df_mic,'mic_log_above_dl',2],
    'tn'   :[df_clean,'TN_tn_log',5],
    'tp'   :[df_clean,'TP_tp_log',5],
}


In [3]:
print(df_clean.columns.values)

['chla_log' 'cybv_log' 'cypro' 'mic_log_above_dl' 'DOY_COL' 'E_pr_gt90'
 'E_pr_gt90_jja' 'E_pr_gt90_mam' 'E_pr_gt95' 'E_pr_gt95_jja'
 'E_pr_gt95_mam' 'E_pr_gt99' 'E_pr_gt99_jja' 'E_pr_gt99_mam' 'MONTH_COL'
 'TN_tn_log' 'TN_tn_ugl' 'TP_tp_log' 'TP_tp_ugl' 'airtemp_ann'
 'airtemp_jja' 'airtemp_mam' 'area_ha' 'area_ha_log' 'cur_NANI'
 'cur_NANI_FERT' 'cur_NANI_FERT_log' 'cur_NANI_arcsinh' 'cur_NAPI'
 'cur_NAPI_FERT' 'cur_NAPI_FERT_log' 'cur_NAPI_arcsinh' 'depth_m'
 'depth_m_log' 'flux_NFERTxE_pr_gt90' 'flux_NFERTxE_pr_gt95'
 'flux_NFERTxE_pr_gt99' 'flux_NFERTxprec_tot_ann_mm'
 'flux_NFERTxprec_tot_ann_mm_log' 'flux_PFERTxE_pr_gt90'
 'flux_PFERTxE_pr_gt95' 'flux_PFERTxE_pr_gt99' 'flux_PFERTxprec_tot_ann_mm'
 'flux_PFERTxprec_tot_ann_mm_log' 'flux_Q_TN' 'flux_lnQ_TN' 'lat_dd'
 'lon_dd' 'pdsi_ann' 'pdsi_jja' 'pdsi_mam' 'prec_tot_ann_mm'
 'prec_tot_ann_mm_log' 'prec_tot_jja_mm' 'prec_tot_jja_mm_log'
 'prec_tot_mam_mm' 'prec_tot_mam_mm_log' 'strat_buoy_freq_s2'
 'strat_surfminbot' 'strat_therm

In [7]:
# Basic test to explain colinearity to reviewer #2

for whichmodel in ['chla']:#,'cybv','mic','tn','tp']:
    print(whichmodel)
    df = dict_meta[whichmodel][0]

    print(len(df))
    df = df.groupby(level=[0,1]).agg('mean')
    print(len(df))

    best_models = pd.DataFrame.from_csv(
        'best_models/best_models_{}_{}_{}noxes.csv'.format(whichmodel,whichyear,suff))
    best_models = best_models.loc[best_models['n']==dict_meta[whichmodel][2]]
    best_ols_vars = best_models.iloc[0].model
    best_ols_vars = ast.literal_eval(best_ols_vars) #need literal eval because otherwise is str
    best_ols_vars = list(best_ols_vars)
    print(best_ols_vars)

    #get regressionresults object
    def get_mod(i_vars,dafr):
        df_i = dafr[[dict_meta[whichmodel][1]]+i_vars].copy()
        y = df_i[dict_meta[whichmodel][1]]
        X = df_i[i_vars]
        X = sm.add_constant(X)
        est = sm.OLS(y,X,missing='raise').fit() 
        return [est,df_i]

    print('R2 with TN alone:')
    [best_ols,best_ols_df] = get_mod(['TN_tn_log'],df)
#     print(best_ols.summary())
    print(best_ols.rsquared)
    print('R2 with TN and TP together')
    [best_ols,best_ols_df] = get_mod(['TN_tn_log','TP_tp_log'],df)
#     print(best_ols.summary())
    print(best_ols.rsquared)
    print('Pearson correlation coefficient r with TN and TP and 2-tailed t-test p-value')
    print(stats.stats.pearsonr(df['TN_tn_log'],df['TP_tp_log']))
    
    est = sm.OLS(df['TP_tp_log'],sm.add_constant(df['TN_tn_log'])).fit()
    print(est.summary())
#     print(est.rsquared)
    df['TP_uncorrelated'] = df['TP_tp_log'] - est.predict()
    print('Pearson correlation coefficient r with TN and TP residual and 2-tailed t-test p-value')
    print(stats.stats.pearsonr(df['TN_tn_log'],df['TP_uncorrelated']))
    
    print('R2 with TN and TP uncorrelated together')
    [best_ols,best_ols_df] = get_mod(['TN_tn_log','TP_uncorrelated'],df)
#     print(best_ols.summary())
    print(best_ols.rsquared)
    

chla
1261
1254
['TN_tn_log', 'TP_tp_log', 'airtemp_jja', 'prec_tot_ann_mm_log']
R2 with TN alone:
0.57741587891
R2 with TN and TP together
0.636246582774
Pearson correlation coefficient r with TN and TP and 2-tailed t-test p-value
(0.79500661438590292, 4.4358140791003639e-274)
                            OLS Regression Results                            
Dep. Variable:              TP_tp_log   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     2151.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):          4.44e-274
Time:                        17:40:08   Log-Likelihood:                -1654.5
No. Observations:                1254   AIC:                             3313.
Df Residuals:                    1252   BIC:                             3323.
Df Model:                           1                                         
Covariance 

In [8]:
# correction for duplicates?

for whichmodel in ['chla','cybv','mic','tn','tp']:
    print(whichmodel)
    df = dict_meta[whichmodel][0]

    print(len(df))
    df = df.groupby(level=[0,1]).agg('mean')
    print(len(df))

    best_models = pd.DataFrame.from_csv(
        'best_models/best_models_{}_{}_{}noxes.csv'.format(whichmodel,whichyear,suff))
    best_models = best_models.loc[best_models['n']==dict_meta[whichmodel][2]]
    best_ols_vars = best_models.iloc[0].model
    best_ols_vars = ast.literal_eval(best_ols_vars) #need literal eval because otherwise is str
    best_ols_vars = list(best_ols_vars)
    print(best_ols_vars)

    #get regressionresults object
    def get_mod(i_vars,dafr):
        df_i = dafr[[dict_meta[whichmodel][1]]+i_vars].copy()
        y = df_i[dict_meta[whichmodel][1]]
        X = df_i[i_vars]
        X = sm.add_constant(X)
        est = sm.OLS(y,X,missing='raise').fit() 
        return [est,df_i]

    [best_ols,best_ols_df] = get_mod(best_ols_vars,df)
    print(best_ols.summary())
    print(best_ols.rsquared)

chla
1261
1254
['TN_tn_log', 'TP_tp_log', 'airtemp_jja', 'prec_tot_ann_mm_log']
                            OLS Regression Results                            
Dep. Variable:               chla_log   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     689.2
Date:                Sat, 21 Dec 2019   Prob (F-statistic):          3.51e-314
Time:                        17:40:14   Log-Likelihood:                -1604.4
No. Observations:                1254   AIC:                             3219.
Df Residuals:                    1249   BIC:                             3244.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------

In [14]:
# R2 in un-transformed space

for whichmodel in ['chla','cybv','mic','tn','tp']:
    print(whichmodel)
    df = dict_meta[whichmodel][0]

    print(len(df))
    df = df.groupby(level=[0,1]).agg('mean')
    print(len(df))

    best_models = pd.DataFrame.from_csv(
        'best_models/best_models_{}_{}_{}noxes.csv'.format(whichmodel,whichyear,suff))
    best_models = best_models.loc[best_models['n']==dict_meta[whichmodel][2]]
    best_ols_vars = best_models.iloc[0].model
    best_ols_vars = ast.literal_eval(best_ols_vars) #need literal eval because otherwise is str
    best_ols_vars = list(best_ols_vars)
    print(best_ols_vars)

    #get regressionresults object
    def get_mod(i_vars,dafr):
        df_i = dafr[[dict_meta[whichmodel][1]]+i_vars].copy()
        y = df_i[dict_meta[whichmodel][1]]
        X = df_i[i_vars]
        X = sm.add_constant(X)
        est = sm.OLS(y,X,missing='raise').fit() 
        return [est,df_i]

    [best_ols,best_ols_df] = get_mod(best_ols_vars,df)
    print(best_ols.summary())
    print(best_ols.rsquared)
    
    predicted = np.exp(best_ols.predict())
    true_y = np.exp(df[dict_meta[whichmodel][1]])
    [slope, intercept, r_value, p_value, std_err] = stats.linregress(predicted,true_y)
    print("r-squared in un-transformed space: %f" % r_value**2)
    

chla
1261
1254
['TN_tn_log', 'TP_tp_log', 'airtemp_jja', 'prec_tot_ann_mm_log']
                            OLS Regression Results                            
Dep. Variable:               chla_log   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     689.2
Date:                Sun, 03 Mar 2019   Prob (F-statistic):          3.51e-314
Time:                        23:34:36   Log-Likelihood:                -1604.4
No. Observations:                1254   AIC:                             3219.
Df Residuals:                    1249   BIC:                             3244.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------

In [9]:
#Summary statistics for Table 1
print(len(df))
for varname in ['chla_log','cybv_log','mic_log_above_dl','TN_tn_ugl','TP_tp_ugl',
                'area_ha','depth_m','temp_surf','temp_mean']:
    if 'log' in varname:
        print(np.exp(df[varname]).describe()[['min','max','mean','50%']])
    else:
        print(df[varname].describe()[['min','max','mean','50%']])
    print('')

1260
min       0.070000
max     936.000000
mean     31.567003
50%       8.440000
Name: chla_log, dtype: float64

min     1.934490e+01
max     1.718506e+08
mean    3.120270e+06
50%     2.598560e+05
Name: cybv_log, dtype: float64

min       0.100000
max     192.000000
mean      2.274175
50%       0.376430
Name: mic_log_above_dl, dtype: float64

min         5.000000
max     54000.000000
mean     1271.121667
50%       616.000000
Name: TN_tn_ugl, dtype: float64

min        1.000000
max     3636.000000
mean     109.022381
50%       33.000000
Name: TP_tp_ugl, dtype: float64

min         1.084365
max     57597.959320
mean      742.564215
50%        50.235671
Name: area_ha, dtype: float64

min      0.500000
max     60.300000
mean     8.447905
50%      5.200000
Name: depth_m, dtype: float64

min     10.000000
max     34.505000
mean    24.957119
50%     25.200000
Name: temp_surf, dtype: float64

min      7.043750
max     34.058000
mean    22.066550
50%     22.641667
Name: temp_mean, dtype: float6